In [1]:
import pandas as pd
import numpy as np
import dask #used here to scale computing capabilities of pandas through parallelism
import dask.dataframe as dd
import pickle #since openning the data was time consuming let's just save it in a pickle file

In [2]:
crsp=dd.read_csv("CRSP_M_19502020.csv")

In [3]:
ibes=dd.read_csv("IBES_Q_19802020.csv")

In [4]:
comp=dd.read_csv("Compustat_Q_19712020.csv")

In [5]:
drp_crsp=['DIVAMT','EXCHCD','FACPR', 'NWPERM', 'BIDLO', 'ASKHI','TICKER','BID','ASK','SPREAD','RETX','SHRCD','Unnamed: 0']
crsp=crsp.drop(columns=drp_crsp)

In [6]:
crsp

,PERMNO,date,NCUSIP,CUSIP,PRC,VOL,RET,SHROUT
npartitions=12,,,,,,,,
,int64,object,object,object,float64,float64,float64,float64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [7]:
crsp['PRC']=crsp['PRC'].abs()

In [8]:
drop_comp=['indfmt', 'consol','curcdq','popsrc','datafmt','tic','datacqtr', 'datafqtr', 'actq','ceqq','dlcq', 'dlttq','exchg', 'costat']
comp=comp.drop(columns=drop_comp)

In [9]:
comp

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
npartitions=2,,,,,,,,,,,,,,
,int64,int64,object,int64,int64,int64,object,float64,float64,float64,float64,float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
#We have to drop he last digit in CUSIP for Compustat to be equal to CRSP
comp['cusip'] = comp['cusip'].str[:-1]
comp

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
npartitions=2,,,,,,,,,,,,,,
,int64,int64,object,int64,int64,int64,object,float64,float64,float64,float64,float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
drop_ibes=['Unnamed: 0','ESTIMATOR','FPI','REVDATS','REVTIMS']
ibes=ibes.drop(columns=drop_ibes)
ibes

,TICKER,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
npartitions=17,,,,,,,,,,
,object,object,int64,float64,object,object,object,float64,object,object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [12]:
#Celaning out the date that comes as a data reading error when we read a stata file
ibes['ANNTIMS']=ibes['ANNTIMS'].str[10:]
ibes['ANNTIMS_ACT']=ibes['ANNTIMS_ACT'].str[10:]
ibes

,TICKER,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
npartitions=17,,,,,,,,,,
,object,object,int64,float64,object,object,object,float64,object,object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [13]:
#transform both columns into datetime.date objects
ibes['ANNDATS']=dd.to_datetime(ibes['ANNDATS'])
ibes['ANNDATS_ACT']=dd.to_datetime(ibes['ANNDATS_ACT'])
ibes['FPEDATS']=dd.to_datetime(ibes['FPEDATS'])

In [14]:
#the date.days attribute will return the days in int format
ibes_fltr = ibes[(ibes['ANNDATS']-ibes['ANNDATS_ACT']).dt.days >= -15]

In [15]:
ibes_fltr_grp=ibes_fltr.groupby(['TICKER','FPEDATS','ANALYS'])
ibes_fin_fltr=ibes_fltr_grp['ANNDATS'].max()
ibes_fin_fltr

Dask Series Structure:
npartitions=1
    datetime64[ns]
               ...
Name: ANNDATS, dtype: datetime64[ns]
Dask Name: series-groupby-max-agg, 412 tasks

In [16]:
df1=ibes_fin_fltr.reset_index()

In [17]:
df2=df1.merge(ibes_fltr)

In [18]:
df2

,TICKER,FPEDATS,ANALYS,ANNDATS,CUSIP,VALUE,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
npartitions=17,,,,,,,,,,
,object,datetime64[ns],int64,datetime64[ns],object,float64,object,float64,datetime64[ns],object
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [19]:
df2=df2.compute()

In [20]:
ibes_fnl=df2.groupby(['TICKER','CUSIP','FPEDATS','ANNDATS_ACT','ANNTIMS_ACT','ACTUAL']).agg({'VALUE':'median'})

In [21]:
ibes_fnl

VALUE
TICKER CUSIP    FPEDATS    ANNDATS_ACT ANNTIMS_ACT ACTUAL       
0000   87482X10 2014-03-31 2014-05-06   10:45:00    0.12   0.100
                2014-06-30 2014-08-06   17:05:00    0.27   0.190
0001   26878510 2013-12-31 2014-02-27   23:13:00    0.20   0.230
                2014-03-31 2014-05-07   16:15:00    0.18   0.170
                2014-06-30 2014-08-06   17:10:00    0.23   0.200
...                                                          ...
ZYNE   98986X10 2019-03-31 2019-05-08   06:51:00   -0.47  -0.530
                2021-03-31 2021-05-12   06:45:00   -0.20  -0.190
ZYNX   98986M10 2019-12-31 2020-02-27   16:02:00    0.09   0.080
                2020-06-30 2020-07-28   16:01:00    0.09   0.096
                2020-09-30 2020-10-27   16:01:00    0.04   0.035

[178633 rows x 1 columns]

In [22]:
ibes_fnl=ibes_fnl.reset_index()
ibes_fnl

,TICKER,CUSIP,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.100
1,0000,87482X10,2014-06-30,2014-08-06,17:05:00,0.27,0.190
2,0001,26878510,2013-12-31,2014-02-27,23:13:00,0.20,0.230
3,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.18,0.170
4,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.23,0.200
...,...,...,...,...,...,...,...
178628,ZYNE,98986X10,2019-03-31,2019-05-08,06:51:00,-0.47,-0.530
178629,ZYNE,98986X10,2021-03-31,2021-05-12,06:45:00,-0.20,-0.190
178630,ZYNX,98986M10,2019-12-31,2020-02-27,16:02:00,0.09,0.080
178631,ZYNX,98986M10,2020-06-30,2020-07-28,16:01:00,0.09,0.096


In [23]:
ibes_fnl['SURPRISE']=ibes_fnl['ACTUAL']-ibes_fnl['VALUE']
ibes_fnl

,TICKER,CUSIP,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.100,0.020
1,0000,87482X10,2014-06-30,2014-08-06,17:05:00,0.27,0.190,0.080
2,0001,26878510,2013-12-31,2014-02-27,23:13:00,0.20,0.230,-0.030
3,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.18,0.170,0.010
4,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.23,0.200,0.030
...,...,...,...,...,...,...,...,...
178628,ZYNE,98986X10,2019-03-31,2019-05-08,06:51:00,-0.47,-0.530,0.060
178629,ZYNE,98986X10,2021-03-31,2021-05-12,06:45:00,-0.20,-0.190,-0.010
178630,ZYNX,98986M10,2019-12-31,2020-02-27,16:02:00,0.09,0.080,0.010
178631,ZYNX,98986M10,2020-06-30,2020-07-28,16:01:00,0.09,0.096,-0.006


In [27]:
#ibes_filtered=open("ibes_filtered.pickle","wb")

In [28]:
#pickle.dump(ibes_fnl,ibes_filtered)
#ibes_filtered.close()

In [29]:
#ibes_fltr_dask=open("ibes_filtered_dask.pickle","wb")

In [31]:
#sd=dd.from_pandas(ibes_fnl,npartitions=15)

In [32]:
#pickle.dump(sd,ibes_fltr_dask)

In [33]:
#ibes_fltr_dask.close()

In [24]:
crsp=crsp.compute()

/srv/conda/envs/saturn/lib/python3.7/site-packages/dask/core.py:121: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


In [25]:
crsp

,PERMNO,date,NCUSIP,CUSIP,PRC,VOL,RET,SHROUT
0,10000,1985-12-31,NaN,68391610,NaN,NaN,NaN,NaN
1,10000,1986-01-31,68391610,68391610,4.375000,1771.0,NaN,3680.0
2,10000,1986-02-28,68391610,68391610,3.250000,828.0,-0.257143,3680.0
3,10000,1986-03-31,68391610,68391610,4.437500,1078.0,0.365385,3680.0
4,10000,1986-04-30,68391610,68391610,4.000000,957.0,-0.098592,3793.0
...,...,...,...,...,...,...,...,...
246163,93436,2020-08-31,88160R10,88160R10,498.320007,4051970.0,0.741452,931809.0
246164,93436,2020-09-30,88160R10,88160R10,429.010010,17331954.0,-0.139087,948000.0
246165,93436,2020-10-30,88160R10,88160R10,388.040009,8330610.0,-0.095499,947901.0
246166,93436,2020-11-30,88160R10,88160R10,567.599976,7811501.0,0.462736,947901.0


In [36]:
#crsp_filtered=open("crsp_filtered_dask.pickle","wb")

In [37]:
#pickle.dump(dd.from_pandas(crsp,npartitions=15),crsp_filtered)

In [38]:
#crsp_filtered.close()

In [39]:
#comp_filtered=open("comp_filtered_dask.pickle","wb")

In [41]:
#pickle.dump(comp,comp_filtered)

In [42]:
#comp_filtered.close()

In [43]:
ibes_crsp=ibes_fnl.merge(crsp,left_on='CUSIP', right_on='NCUSIP')

In [44]:
ibes_crsp

,TICKER,CUSIP_x,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE,PERMNO,date,NCUSIP,CUSIP_y,PRC,VOL,RET,SHROUT
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-02-28,87482X10,87482X10,13.880000,109117.0,NaN,69943.0
1,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-03-31,87482X10,87482X10,14.640000,78851.0,0.054755,69962.0
2,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-04-30,87482X10,87482X10,13.410000,47374.0,-0.084016,69978.0
3,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-05-30,87482X10,87482X10,13.570000,33556.0,0.011931,69992.0
4,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-06-30,87482X10,87482X10,13.790000,40869.0,0.016212,70451.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345437,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-08-31,48349710,48349710,13.100000,14588.0,0.332655,17880.0
1345438,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-09-30,48349710,48349710,12.590000,9333.0,-0.038931,17908.0
1345439,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-10-30,48349710,48349710,17.209999,23210.0,0.366958,17916.0
1345440,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-11-30,48349710,48349710,18.709999,22268.0,0.087159,17916.0


In [42]:
comp=dd.read_csv("Compustat_Q_19712020.csv",dtype={'exchg': 'float64',
       'naics': 'float64',
       'sic': 'float64'})
drop_comp=['Unnamed: 0','indfmt', 'consol','curcdq','popsrc','datafmt','tic','datacqtr', 'datafqtr', 'actq','ceqq','dlcq', 'dlttq','exchg', 'costat']
comp=comp.drop(columns=drop_comp)
comp['cusip'] = comp['cusip'].str[:-1]
comp


,gvkey,datadate,fyearq,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
npartitions=2,,,,,,,,,,,,,
,int64,object,int64,int64,int64,object,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [43]:
comp=comp.compute()

In [44]:
comp['datadate']=pd.to_datetime(comp['datadate'])

In [45]:
ibes_comp=pd.merge(ibes_fnl,comp,left_on=["FPEDATS","CUSIP"],right_on=["datadate","cusip"])

In [46]:
ibes_comp

,TICKER,CUSIP,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE,gvkey,datadate,...,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
0,0001,26878510,2013-12-31,2014-02-27,23:13:00,0.20,0.230,-0.030,18660,2013-12-31,...,4,12,26878510,8366.000,250.033,43.000,492.000,NaN,211120.0,1311.0
1,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.18,0.170,0.010,18660,2014-03-31,...,1,12,26878510,8601.000,244.900,-99.000,376.000,19.57,211120.0,1311.0
2,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.23,0.200,0.030,18660,2014-06-30,...,2,12,26878510,8935.000,245.800,-113.000,276.000,23.05,211120.0,1311.0
3,0001,26878510,2014-09-30,2014-11-04,16:15:00,0.24,0.215,0.025,18660,2014-09-30,...,3,12,26878510,9026.000,245.800,306.000,953.000,17.48,211120.0,1311.0
4,0001,26878510,2014-12-31,2015-02-18,16:22:00,0.22,0.240,-0.020,18660,2014-12-31,...,4,12,26878510,10219.000,245.800,633.000,1479.000,10.44,211120.0,1311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74069,ZYNE,98986X10,2019-03-31,2019-05-08,06:51:00,-0.47,-0.530,0.060,25128,2019-03-31,...,1,12,98986X10,75.517,21.075,-9.147,0.000,5.42,325412.0,2834.0
74070,ZYNE,98986X10,2021-03-31,2021-05-12,06:45:00,-0.20,-0.190,-0.010,25128,2021-03-31,...,1,12,98986X10,108.808,41.252,-7.962,0.000,4.65,325412.0,2834.0
74071,ZYNX,98986M10,2019-12-31,2020-02-27,16:02:00,0.09,0.080,0.010,129441,2019-12-31,...,4,12,98986M10,28.277,32.792,2.947,14.162,7.87,334510.0,3845.0
74072,ZYNX,98986M10,2020-06-30,2020-07-28,16:01:00,0.09,0.096,-0.006,129441,2020-06-30,...,2,12,98986M10,36.759,33.335,3.017,19.263,24.87,334510.0,3845.0


In [48]:
crsp['date']=pd.to_datetime(crsp['date'])

In [49]:
ibes_crsp=pd.merge(ibes_fnl,crsp,left_on=['FPEDATS','CUSIP'], right_on=['date','NCUSIP'])

In [50]:
ibes_crsp

,TICKER,CUSIP_x,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE,PERMNO,date,NCUSIP,CUSIP_y,PRC,VOL,RET,SHROUT
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.1200,0.1000,0.0200,14471,2014-03-31,87482X10,87482X10,14.640000,78851.0,0.054755,69962.0
1,0000,87482X10,2014-06-30,2014-08-06,17:05:00,0.2700,0.1900,0.0800,14471,2014-06-30,87482X10,87482X10,13.790000,40869.0,0.016212,70451.0
2,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.1800,0.1700,0.0100,14392,2014-03-31,26878510,26878510,19.570000,186376.0,0.050456,243878.0
3,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.2300,0.2000,0.0300,14392,2014-06-30,26878510,26878510,23.049999,261603.0,0.149053,244979.0
4,0001,26878510,2014-09-30,2014-11-04,16:15:00,0.2400,0.2150,0.0250,14392,2014-09-30,26878510,26878510,17.480000,224309.0,-0.095706,244940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119544,ZY,98919510,1986-07-31,1986-08-13,00:00:00,0.0047,0.0147,-0.0100,40539,1986-07-31,98919510,87254010,34.375000,41681.0,-0.198251,58964.0
119545,ZY,98919510,1986-10-31,1986-11-10,00:00:00,0.0119,0.0056,0.0063,40539,1986-10-31,98919510,87254010,24.625000,89710.0,0.013538,58964.0
119546,ZYNX,98986M10,2019-12-31,2020-02-27,16:02:00,0.0900,0.0800,0.0100,18418,2019-12-31,98986M10,98986M10,7.870000,64260.0,-0.250476,32792.0
119547,ZYNX,98986M10,2020-06-30,2020-07-28,16:01:00,0.0900,0.0960,-0.0060,18418,2020-06-30,98986M10,98986M10,24.870001,311692.0,0.281959,33410.0


In [51]:
ibes_crsp_comp=pd.merge(ibes_crsp,comp,left_on=['FPEDATS','CUSIP_y'],right_on=['datadate','cusip'])

In [53]:
ibes_crsp_comp

,TICKER,CUSIP_x,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE,PERMNO,date,...,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
0,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.1800,0.1700,0.0100,14392,2014-03-31,...,1,12,26878510,8601.000,244.900,-99.000,376.000,19.570,211120.0,1311.0
1,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.2300,0.2000,0.0300,14392,2014-06-30,...,2,12,26878510,8935.000,245.800,-113.000,276.000,23.050,211120.0,1311.0
2,0001,26878510,2014-09-30,2014-11-04,16:15:00,0.2400,0.2150,0.0250,14392,2014-09-30,...,3,12,26878510,9026.000,245.800,306.000,953.000,17.480,211120.0,1311.0
3,0001,26878510,2014-12-31,2015-02-18,16:22:00,0.2200,0.2400,-0.0200,14392,2014-12-31,...,4,12,26878510,10219.000,245.800,633.000,1479.000,10.440,211120.0,1311.0
4,0001,26878510,2015-03-31,2015-04-29,16:15:00,0.1300,0.1255,0.0045,14392,2015-03-31,...,1,12,26878510,10313.000,248.800,19.000,493.000,10.480,211120.0,1311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62386,ZY,98919510,1986-07-31,1986-08-13,00:00:00,0.0047,0.0147,-0.0100,40539,1986-07-31,...,2,1,87254010,1940.132,58.988,9.042,1212.672,34.375,448140.0,5651.0
62387,ZY,98919510,1986-10-31,1986-11-10,00:00:00,0.0119,0.0056,0.0063,40539,1986-10-31,...,3,1,87254010,2259.815,59.006,22.876,1316.424,24.625,448140.0,5651.0
62388,ZYNX,98986M10,2019-12-31,2020-02-27,16:02:00,0.0900,0.0800,0.0100,18418,2019-12-31,...,4,12,98986M10,28.277,32.792,2.947,14.162,7.870,334510.0,3845.0
62389,ZYNX,98986M10,2020-06-30,2020-07-28,16:01:00,0.0900,0.0960,-0.0060,18418,2020-06-30,...,2,12,98986M10,36.759,33.335,3.017,19.263,24.870,334510.0,3845.0


In [54]:
final_df=open("ibes_crsp_comp.pickle","wb")
pickle.dump(ibes_crsp_comp,final_df)
final_df.close()